# ***Lesson 0 - Pre-Coding Knowledge***

# PART 1 - What Python Actually is

## 1.1 Python is a Language, Not a Program

When people say "Python", they are often referring to two different things:

1. The Python **language**
2. A Python **implementation**

These are related, but not the same.

---

### Think of it like this

- English = the language
- People = implementations of the language

Different people may speak English differently, but the underlying grammar rules remain the same.

---

### Interpreted vs Compiled

Python is an **interpreted** language, not a compiled one.
The language itself defines *what* the rules are, not *how* they are executed.

---

### Key idea

***CPython is the most common implementation that “speaks” the Python language.***

<img src="https://files.realpython.com/media/A-Tourist-Guide-to-the-CPython-Source-Code_Watermarked.5f05a87331ac.jpg" width="300">

## 1.2 CPython

When you type:

```bash
python my_program.py
```

---

You are actually running a *C* program that:

- Reads Python source code
- Converts it into instructions
- Executes them

---

The *C* program:

- Manages memory
- Handles imports
- Executes the bytecode
- Enforces the *GIL*

---

This *C* program is the Python *interpreter*, it interprets the python byte code and executes it.

---

### Here is the way Python is interpreted in CPython:

<img src="https://www.c-sharpcorner.com/article/why-learn-python-an-introduction-to-python/Images/last2.png" width="400">


## 1.3 CPython Alternatives

Just as there is **CPython**, there are other Python implementations designed to solve different problems or target different platforms.

### Alternative Python implementations

These are full Python implementations with their own interpreters and runtimes:

- **PyPy**
  Focuses on performance using JIT (Just-In-Time) compilation.

- **Jython**
  Runs Python on the JVM, enabling tight integration with Java libraries.

- **IronPython**
  Runs Python on the .NET runtime with strong .NET interoperability.

- **MicroPython**
  A lightweight Python implementation for embedded systems and microcontrollers.

- **GraalPy**
  Python integrated with GraalVM, designed for polyglot applications.

---

### CPython performance alternatives

The following tools do **not** replace CPython.
Instead, they change *how code is executed* or *how fast it runs* while still relying on CPython:

- **Cython**
  Translates Python-like code into C to produce faster native extensions.

- **Numba**
  Applies JIT compilation to numerical Python code, especially with NumPy.

- **Pyston**
  A performance-focused fork of CPython that optimizes execution.

---

### Key distinction

> Some projects **replace the Python interpreter**.
> Others **optimize execution while keeping CPython underneath**.

----

<img src="https://media.licdn.com/dms/image/v2/D4D22AQET5Wv1_jiKAw/feedshare-shrink_800/B4DZX0BwpdG8Ak-/0/1743555859683?e=2147483647&v=beta&t=DXtpqRewj595P4a7j4Cowv5pVcz-Zw-cWWcck5fDiIw" width="300">

# PART 2 - How Python runs

## 2.1 Python does NOT run code line-by-line

This is a common misconception/simplification about Python. It goes through multiple steps before being executed.

When Python sees this:
```python
x = 1 + 2
```

It does *not* immediately execute it. Instead, it has 3 major steps.

## 2.2 Step 1 - Parsing (Syntax Check)

Python reads your file and checks for:
- Are keywords used correctly
- Is syntax valid

```python
x = 1 +
```
This would fail.

### This phase creates an Abstract Syntax Tree (AST)

This is a *tree-structured, in-memory representation of your source code's syntax structure*. Its produced after parsing and before bytecode generation.

```
Python source (.py)
        ↓
Lexer (tokens)
        ↓
Parser
        ↓
AST  ←───── you are here
        ↓
Bytecode compiler
        ↓
Code objects
        ↓
Python Virtual Machine
```

In [2]:
import ast
print(ast.dump(ast.parse("x = 1 + 2"), indent=2))

Module(
  body=[
    Assign(
      targets=[
        Name(id='x', ctx=Store())],
      value=BinOp(
        left=Constant(value=1),
        op=Add(),
        right=Constant(value=2)))],
  type_ignores=[])


## 2.3 Step 2 - Compilation to Bytecode

Next it converts the AST into *bytecode*

Bytecode is a low-level platform-independent representation of your python code that the interpreter runs.

***Python is compiled not to machine code, but to byte code***

We can look at it like this:

In [2]:
import dis

def add(a, b):
    return a + b

dis.dis(add)

  4           0 LOAD_FAST                0 (a)
              2 LOAD_FAST                1 (b)
              4 BINARY_ADD
              6 RETURN_VALUE


## 2.4 Step 3 - Execution by the PVM

Now that we have the bytecode, the Python Virtual Machine executes it.

The bytecode runs inside:
- A *loop written in C*
- That reads instructions one by one
- And manipulates the stack

Push values onto a stack, perform operations, pop results, BOOM code is executed oh yeah.

<img src="https://miro.medium.com/v2/resize:fit:1400/1*uvwXR2gF-GzfKadBcdotrQ.gif" width="500">

# PART 3 - Final topics

## Where is bytecode?

The bytecode lives in memory when we execute programs, but you can also write it to disk as a `.pyc` file.

Usually this is stored under `__pycache__/` if you ever see that, this is where your bytecode is stored. But these are just cache files so they exist just to skip recompilation next time.

The interpreter does *NOT* execute directly from disk, on startup the `.pyc` file is:
- Loaded into memory
- Converted to in-memory bytecode objects
- Executed

**A pycache file will be named `<file name>.cpython-<python version>.pyc`.**

<img src="https://en.curict.com/item/6b/6bbf148/2023-07-21%20163342_e_t.png" width="300">

-----

If you want your bytecode written to disk you can execute it like this:

> python -c my_program.py

But `PYTHONDONTWRITEBYTECODE=1` has to be set

-----

## What actually gets stored in memory?

Python hold code objects containing bytecode instructions and stuff.

In [1]:
def f(x):
    return x + 1

print(f.__code__)

<code object f at 0x10954bb30, file "/var/folders/zz/rjyxt22148s5hcvyh8xrx6dr0000gn/T/ipykernel_67632/3514757808.py", line 1>


This is what the Python virtual machine executes

Here is a mental model for this:
```
.py source
   ↓ compile
in-memory bytecode (code objects)
   ↓ optional cache
.pyc on disk
   ↓
Python VM executes in memory
```